In [1]:
!pwd

/notebooks/notebooks/SnowparkRnD/UDFs


In [2]:
!ls

RunExperiment_Sample_On_Snowpark-Copy1.ipynb  UDF_run1.ipynb  run_experiment.py
RunExperiment_Sample_On_Snowpark.ipynb	      __pycache__


In [3]:
!jupyter nbconvert RunExperiment_Sample_On_Snowpark.ipynb --to python --output run_experiment.py

[NbConvertApp] Converting notebook RunExperiment_Sample_On_Snowpark.ipynb to python
[NbConvertApp] Writing 4775 bytes to run_experiment.py


In [4]:
from run_experiment import run_exp

In [5]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf
import pandas as pd

In [6]:
connection_parameters = {
    "account": "ug94937.us-east4.gcp",
    "user": "ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",  # optional
    "warehouse": "FOSFOR_INSIGHT_WH",  # optional
    "database": "FIRST_DB",  # optional
    "schema": "PUBLIC",  # optional
    }
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

In [13]:
session.add_import("run_experiment.py", import_path=".run_experiment")
session.add_packages("numpy", "pandas", "snowflake", "snowflake-ml-python")

The version of package 'snowflake' in the local environment is 0.8.0, which does not fit the criteria for the requirement 'snowflake'. Your UDF might not work when the package version is different between the server and your local environment.


In [10]:
# def udf_run_exp(sf_pass: str, dataset: str, target: str) -> pd.DataFrame:
#     return run_exp(sf_pass, dataset, target)
from snowflake.snowpark.types import StructType, StructField, IntegerType, StringType, ArrayType

In [15]:
udf_run_exp = udf(
    func=run_exp,
    return_type=StringType(),
    input_types=[StringType(), ArrayType(StringType()),StringType(),StringType()],
    name="udf_run_exp",
    session=session
    )

Failed to execute query [queryID: 01b4a543-0000-655a-0000-576d06bcda2e] 
CREATE
TEMPORARY  FUNCTION  udf_run_exp(arg1 STRING,arg2 ARRAY,arg3 STRING,arg4 STRING)
RETURNS STRING
LANGUAGE PYTHON 
VOLATILE
RUNTIME_VERSION=3.9
IMPORTS=('@"FIRST_DB"."PUBLIC".SNOWPARK_TEMP_STAGE_BWX4G2VX14/47760428a0223696b6cbc0ffd5f509d75308be46f9777cdc77be633332f914e5/run_experiment.py.zip')
PACKAGES=('numpy','pandas','snowflake','snowflake-ml-python','cloudpickle==2.0.0')


HANDLER='compute'

AS $$
import pickle

func = pickle.loads(bytes.fromhex('8005951e000000000000008c0e72756e5f6578706572696d656e74948c0772756e5f6578709493942e'))
# The following comment contains the source code generated by snowpark-python for explanatory purposes.
# def run_exp(sf_pass, algos, dataset, target):    
#     import os, importlib
#     from snowflake.snowpark import Session
#     from snowflake.ml.modeling.pipeline import Pipeline
#     from snowflake.ml.modeling.preprocessing import MinMaxScaler, OrdinalEncoder
#     from s

SnowparkSQLException: (1304): 01b4a543-0000-655a-0000-576d06bcda2e: 002002 (42710): SQL compilation error:
Object 'UDF_RUN_EXP' already exists.

In [24]:
algos=['snowflake.ml.modeling.naive_bayes.GaussianNB',
       'snowflake.ml.modeling.neighbors.KNeighborsClassifier',
      ]

In [14]:
out = session.sql(f"select udf_run_exp('Enlightme#2024',ARRAY_CONSTRUCT('snowflake.ml.modeling.naive_bayes.GaussianNB'),'EMPLOYEE', 'LEAVEORNOT') as run").collect()

SnowparkSQLException: (1304): 01b4a543-0000-658c-0000-576d06bce58a: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 146, in compute
  File "_udf_code.py", line 135, in wrapper
  File "/home/udf/1466800081/run_experiment.py.zip/run_experiment.py", line 3, in run_exp
    from snowflake.snowpark import Session
ModuleNotFoundError: No module named 'snowflake'
 in function UDF_RUN_EXP with handler compute

In [13]:
@udf
def add_udf(x: int, y: int) -> int:
    return x + y
df = session.create_dataframe([[1, 2], [3, 4]], schema=["x", "y"])
df.select(add_udf("x", "y")).to_df("add_result").collect()

[Row(ADD_RESULT=3), Row(ADD_RESULT=7)]

In [7]:
session.close()